jakie są i jak używamy w modelu

In [1]:
import sklearn.metrics
import scipy.special
import sklearn.datasets as ds
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor, LogisticRegression
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
import keras.losses
import keras.backend

W bibliotece scikit-learn funkcje straty, które używane są także jako metryki, można znaleźć w sekcji "sklearn.metrics: Metrics" (https://scikit-learn.org/stable/modules/classes.html?highlight=metrics#module-sklearn.metrics)

Jeśli chodzi o ich wykorzystanie w modelach, często parametr nazwany "loss" odpowiada za wybór funkcji straty - wówczas dostępnych funkcji straty należy szukać w dokumentacji konkretnego modelu

### dla problemu regresji

##### - jakie są

poniżej przedstawiono dostępne implementacje funkcji straty, niestety w formie tak jakby pełniły rolę metryk, porównujących predykcję z rzeczywistymi wartościami

In [2]:
dummy_y_true = np.array([[0.35], [0.14]])
dummy_y_pred = np.array([[0.26], [0.09]])

# z sklearn:
mean_absolute_error = sklearn.metrics.mean_absolute_error(dummy_y_true, dummy_y_pred)
mean_squared_error = sklearn.metrics.mean_squared_error(dummy_y_true, dummy_y_pred)
root_mean_squared_error = sklearn.metrics.mean_squared_error(dummy_y_true, dummy_y_pred, squared=False)
mean_squared_log_error = sklearn.metrics.mean_squared_log_error(dummy_y_true, dummy_y_pred)
median_absolute_error = sklearn.metrics.median_absolute_error(dummy_y_true, dummy_y_pred)
mean_absolute_percentage_error = sklearn.metrics.mean_absolute_percentage_error(dummy_y_true, dummy_y_pred)
mean_poisson_deviance = sklearn.metrics.mean_poisson_deviance(dummy_y_true, dummy_y_pred)
mean_gamma_deviance = sklearn.metrics.mean_gamma_deviance(dummy_y_true, dummy_y_pred)
mean_tweedie_deviance = sklearn.metrics.mean_tweedie_deviance(dummy_y_true, dummy_y_pred)
mean_pinball_loss = sklearn.metrics.mean_pinball_loss(dummy_y_true, dummy_y_pred)
cosine_similarity = sklearn.metrics.pairwise.cosine_similarity(dummy_y_true, dummy_y_pred)

# z scipy:
huber_loss = scipy.special.huber(dummy_y_true, dummy_y_pred)

# implementacja funkcji log_cosh występuje "niestety" tylko w bibliotece Keras, która nie jest w zakresie tego kursu,
# natomiast jej działanie jak najbardziej można wykorzystać także i tutaj
log_cosh = keras.losses.log_cosh(dummy_y_true, dummy_y_pred).numpy().tolist()
 
print('mean_absolute_error:', mean_absolute_error)
print('mean_squared_error:', mean_squared_error)
print('root_mean_squared_error:', root_mean_squared_error)
print('mean_squared_log_error:', mean_squared_log_error)
print('median_absolute_error:', median_absolute_error)
print('mean_absolute_percentage_error:', mean_absolute_percentage_error)
print('mean_poisson_deviance:', mean_poisson_deviance)
print('mean_gamma_deviance:', mean_gamma_deviance)
print('mean_tweedie_deviance:', mean_tweedie_deviance)
print('mean_pinball_loss:', mean_pinball_loss)
print('cosine_similarity:', cosine_similarity)
print('huber_loss:', huber_loss)
print('log_cosh:', log_cosh)

mean_absolute_error: 0.06999999999999999
mean_squared_error: 0.005299999999999998
root_mean_squared_error: 0.07280109889280517
mean_squared_log_error: 0.003385794800683798
median_absolute_error: 0.06999999999999999
mean_absolute_percentage_error: 0.30714285714285716
mean_poisson_deviance: 0.025894618532841557
mean_gamma_deviance: 0.16262512596243095
mean_tweedie_deviance: 0.005299999999999998
mean_pinball_loss: 0.034999999999999996
cosine_similarity: [[1. 1.]
 [1. 1.]]
huber_loss: [[0.0338 ]
 [0.00405]]
log_cosh: [0.004044542376182347, 0.0012494776089713167]


funkcja straty MBE (Mean Bias Error) nie posiada implementacji w żadnej z przeszukanych bibliotek, ale nie jest trudno ją zaimplementować samemu

In [3]:
def mean_bias_error(y_true, y_pred):
    return np.mean(y_pred - y_true)

print('mean_bias_error:', mean_bias_error(dummy_y_true, dummy_y_pred))

mean_bias_error: -0.06999999999999999


##### - jak używamy w modelu

W scikit-learn istnieją modele, np. dla regresji liniowej (sklearn.linear_model), gdzie funkcja straty jest już z góry wybrana. Dla przykładu:
- LinearRegression to tak na prawdę tylko wrapper na "least squares solver" z biblioteki numpy, który wykorzystuje metodę Ordinary Least Squares
- Ridge - "This model solves a regression model where the loss function is the **linear least squares** function and regularization is given by the l2-norm", minimalizuje funkcję:
$$||y - X_w||^2_2 + alpha * ||w||^2_2$$
- HuberRegressor oparty jest o funkcję straty Hubera
- Istnieje także model SGDRegressor, który wykorzystuje Stochastic Gradient Descent, gdzie wagi są aktualizowane po każdej próbce. Dla tego modelu mamy cztery funkcje straty do wyboru: ‘squared_error’, ‘huber’, ‘epsilon_insensitive’, ‘squared_epsilon_insensitive’

In [4]:
diabetes_ds = ds.load_diabetes()
X = diabetes_ds.data
y = diabetes_ds.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
sgd = SGDRegressor(loss='squared_error', max_iter=10000)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print(y_test.tolist()[:5])
print(y_pred[:5])

[139.0, 72.0, 79.0, 280.0, 150.0]
[182.97485282  84.32127311 109.46137067 218.79134389 109.20079235]


### dla problemu klasyfikacji

##### -jakie są

In [5]:
dummy_binary_y_pred = [1, 1, 0]
dummy_binary_y_true = [1, 0, 0]

# dla dwóch klass
brier_score_loss = sklearn.metrics.brier_score_loss(dummy_binary_y_true, dummy_binary_y_pred)
# dla dwóch lub więcej klas
hamming_loss = sklearn.metrics.hamming_loss(dummy_binary_y_true, dummy_binary_y_pred)
# dla dwóch lub więcej klas
hinge_loss = sklearn.metrics.hinge_loss(dummy_binary_y_true, dummy_binary_y_pred)
# dla dwóch lub więcej klas
zero_one_loss = sklearn.metrics.zero_one_loss(dummy_binary_y_true, dummy_binary_y_pred)

dummy_y_true = ['spam', 'lettuce', 'lettuce', 'spam']
dummy_onehot_y_pred = [[.1, .9], [.9, .1], [.8, .2], [.35, .65]]

# aka cross-entropy loss - dla dwóch lub więcej klas
log_loss = sklearn.metrics.log_loss(dummy_y_true, dummy_onehot_y_pred)

print('brier_score_loss:', brier_score_loss)
print('hamming_loss:', hamming_loss)
print('hinge_loss:', hinge_loss)
print('log_loss:', log_loss)
print('zero_one_loss:', zero_one_loss)

brier_score_loss: 0.3333333333333333
hamming_loss: 0.3333333333333333
hinge_loss: 1.0
log_loss: 0.21616187468057912
zero_one_loss: 0.33333333333333337


implementacje funkcji straty, które istnieją tylko w bibliotece Keras:

In [6]:
dummy_binary_y_true = keras.backend.variable(np.asarray([1, 1, 0, 0]))
dummy_binary_y_pred = keras.backend.variable(np.asarray([0.8, 0.65, 0.43, 0.12]))

dummy_multi_onehot_y_true = keras.backend.variable(np.asarray([[0, 1, 0], [0, 0, 1]]))
dummy_multi_y_pred = keras.backend.variable(np.asarray([[0.05, 0.95, 0], [0.1, 0.8, 0.1]]))

dummy_multi_y_true = keras.backend.variable(np.asarray([1, 2]))

# dla dwóch klas
squared_hinge = keras.losses.squared_hinge(dummy_binary_y_true, dummy_binary_y_pred)
# dla dwóch lub więcej klas
sparse_categorical_crossentropy = keras.losses.sparse_categorical_crossentropy(dummy_multi_y_true, dummy_multi_y_pred)
# dla dwóch lub więcej klas
poisson = keras.losses.poisson(dummy_multi_onehot_y_true, dummy_multi_y_pred)
# dla dwóch lub więcej klas
kl_divergence = keras.losses.kl_divergence(dummy_multi_onehot_y_true, dummy_multi_y_pred)
# dla dwóch klas
categorical_hinge = keras.losses.categorical_hinge(dummy_binary_y_true, dummy_binary_y_pred)

print('squared_hinge:', squared_hinge.numpy().tolist())
print('sparse_categorical_crossentropy:', sparse_categorical_crossentropy.numpy().tolist())
print('poisson:', poisson.numpy().tolist())
print('kl_divergence:', kl_divergence.numpy().tolist())
print('categorical_hinge:', categorical_hinge.numpy().tolist())

squared_hinge: 0.8654500246047974
sparse_categorical_crossentropy: [0.05129344388842583, 2.3025851249694824]
poisson: [0.35043105483055115, 1.1008614301681519]
kl_divergence: [0.051292017102241516, 2.302582263946533]
categorical_hinge: 0.0


##### -jak używamy w modelu

jeśli chodzi o użycie funkcji straty w dostępnych modelach sklearn, przykłady:
- model LinearSVC, czyli Linear Support Vector Classification, pozwala na wybranie jako funkcji straty jedną z dwóch: hinge lub squared hinge
- model LogisticRegression w wypadku problemu wieloklasowego uzywa funkcji log loss, czyli wspomniana wcześniej cross-entropy
- model SGDClassifier, czyli estymator wykorzystujący SGD, który implementuje różne modele liniowe poprzez kontrolę parametru loss - czyli wybranie odpowiedniej funkcji straty powoduje wybranie konkretnego modelu, np. wybranie funkcji hinge wybiera liniowy model SVM. Dostępne funkcje straty dla SGDClassifier: ‘hinge’, ‘log’, ‘modified_huber’, ‘squared_hinge’, ‘perceptron’

In [7]:
iris = ds.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# categorical cross-entropy loss
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_test.tolist()[:10])
print(y_pred[:10])

[0, 2, 2, 2, 0, 0, 0, 0, 2, 2]
[0 2 2 2 0 0 0 0 2 2]


In [8]:
# wieloklasowy SVM
model = LinearSVC(loss='squared_hinge', dual=False, multi_class='ovr')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_test.tolist()[:10])
print(y_pred[:10])

[0, 2, 2, 2, 0, 0, 0, 0, 2, 2]
[0 2 2 2 0 0 0 0 2 2]
